![title](https://www.measuringu.com/images/puzzle-piece.jpg)

Just like we had to clean data in the last notebook, often when you are working with datasets, you would find that they are not quite...complete. This is because some of the data is missing some features, or simply the feature is not compatible for that particular row of the data. When doing an analysis of the dataset, missing values hurt the results of the overall analysis, since the results would more than likely be modified by a slight or large amount. As a Data Scientist, you'll often  be faced to make a decision of what to do with missing data. The purpose of this notebook is just that, to describe some of the ways you can handle missing data. 

## Checking how much data is missing.

Checking missing data is pretty simple. When analyzing the data, there are several keywords you can lookout for like "None","nan,"NA", etc. On pandas, missing values are denoted as "NaN". Let's load a dataset and check for missing values.

In [1]:
import pandas as pd
titanic = pd.read_csv("titanic.csv")
titanic.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Right of the bat, we can see some missing values in the Age column, and other missing values  in the Cabin  column. Let's check how many there are.

In [2]:
#Find all columns with null values.
#Use the is null method to create a dataframe where nulls are changed to True, then sum all the True values.
#If the total is greater than 0, then the column has missing values.
pd.isnull(titanic).sum() > 0

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

So the Age, Cabin and Embarked columns have missing values.  And it's perfect, since they all describe the 3 most common types of missing data. First, **dealing with missing labels.**

First, let's determine the total of missing values on the Embarked column.

In [3]:
print(sum(pd.isnull(titanic.Embarked)))
print(titanic.Embarked.unique())

2
['S' 'C' 'Q' nan]


So we only have to missing values, and  there are three labels, S C and Q.Now, let's get the count for each label. 

In [4]:
S_Count = sum([1 for embark in titanic.Embarked if embark == "S"])
C_Count = sum([1 for embark in titanic.Embarked if embark == "C"])
Q_Count = sum([1 for embark in titanic.Embarked if embark == "Q"])
print([S_Count,C_Count,Q_Count])

[644, 168, 77]


So the most common label for embarked is "S". Since there aren't many missing values, let's just replace the missing values with the most common label, which is "S". We can use the dataframe.fillna function for this.

In [5]:
titanic.Embarked = titanic.Embarked.fillna("S")

In [6]:
titanic.Embarked.unique()

array(['S', 'C', 'Q'], dtype=object)

And now, we have filled the Embarked column! This is one way to fill missing values on columns that contain labels, by replacing the missing values with the most common value. Now for dealing with missing values on **numeric columns**.

In [7]:
print(sum(pd.isnull(titanic.Age)))


177


There are 177 missing values for the Age. That's much more than what the Embarked column had. Let's get the summary statistics for the Age column, using the **describe** function.

In [8]:
#Prints the summary statistics for a numerical feature, ignoring the null values.
titanic.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

This summary tells us a lot of things. First, the  Age is fairly distributed around the mean age (29), since ranges of values from 25% to 75% of the data is between 20 to 38 years old. Also, the mean and the median are very close, which suggests a normal distribution, in which values are centered around the mean as well. 

That's great and all but how do we fill these missing values?  One option we could use, is replacing all the missing values by the mean age ,29, but since there's so much missing data, this wouldn't be advisable. One alternative is to generate random numbers between the 25% and 75% of the values, which would be closer to the values that the dataset contains. Let's go with that:

In [9]:
#Import numpy for random number generators.
import numpy as np

#Generate random ages.
titanic.Age = titanic.Age.fillna(np.random.randint(20,38))

sum(pd.isnull(titanic.Age))

0

Let's see how this changed the distribution of the Ages, by using **describe** again.

In [10]:
titanic.Age.describe()

count    891.000000
mean      30.752155
std       13.173100
min        0.420000
25%       22.000000
50%       32.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

Since the values that are concentrated near the middle of the distribution increased, the mean and median and the value at the 25% of the Ages increased, while the value at 75% decreased. However, we accomplished filling the missing Age values.  Another better alternative, would be to use predictive modeling based on the other features of the dataset to predict the ages of those that don't have their age registered. After learning about predictive modeling later on, you can revisit this notebook and try to fill missing values using that method. 

Let's take a look at the data one more time:

In [11]:
titanic.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,35.0,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


The only column left with missing data is Cabin,, which appears to be a string type column. There's just **one problem** with this column, and it's as simple as counting how many null values it has.

In [12]:
sum(pd.isnull(titanic.Cabin))

687

Almost **80%** of the values in the Cabin column are missing. When we have so many missing values in a column, there's just no feasible way to fill the missing values in a way that model the actual dataset. In cases like this, most of the time is just better to **drop** the column from the dataframe, and continue without it.

In [13]:
#Drop the Cabin column from the dataset.
titanic = titanic.drop("Cabin",axis = 1)
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


For a final sanity check, let's check if all our columns are free from missing values.

In [14]:
#Verify we don't have any missing values.
pd.isnull(titanic).sum()==  0

PassengerId    True
Survived       True
Pclass         True
Name           True
Sex            True
Age            True
SibSp          True
Parch          True
Ticket         True
Fare           True
Embarked       True
dtype: bool

Perfect. Now this data in a much better state than before. Dealing with these missing values is essential in having a meaningful analysis of the data. It's imperative to completely analyze the data you obtain, so you can purge it of as many imperfections you can find. Another common way to fill missing values, is just to find other occurences similar to the one  that has missing data. This way, you just compare the two, and fill the missing values accordingly.

## Exercise

Fill any missing values from the exercise dataset. It's pretty similar to the Titanic dataset already used earlier.

In [31]:
titanic2 = pd.read_csv("titanic2.csv")
pd.isnull(titanic2).sum()>0

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool

In [32]:
#Your cells below.

## Further reading.

A bit more intuition into missing values: http://www.statisticssolutions.com/missing-values-in-data/